In [36]:
from datetime import datetime, time, date, timedelta
import pytz

In [37]:
sweden_tz = pytz.timezone('Europe/Stockholm')
year = 2025
spring_forward_time = sweden_tz.localize(datetime(year, 3, 31, 2, 0, 0))
fall_back_time =sweden_tz.localize(datetime(year, 10, 27, 3, 0, 0))
wakeup_time = time(7,0,0)
bed_time = time(22,0,0)
work_start_time = time(8,0,0)
work_end_time = time(16,15,0)

print(f"DST starts at: {spring_forward_time} (local time)")
dst_start_utc = spring_forward_time.astimezone(pytz.utc)
print(f"DST starts at: {dst_start_utc} (UTC)")

DST starts at: 2025-03-31 02:00:00+02:00 (local time)
DST starts at: 2025-03-31 00:00:00+00:00 (UTC)


In [38]:
from astral import LocationInfo
location = LocationInfo("Stockholm", "Sweden", "UTC",  59.3293, 18.0686)

In [39]:
from astral.sun import sun
def get_sunrise(specific_date):
    sun_times = sun(location.observer, date=specific_date)
    return sun_times['sunrise']
def get_sunset(specific_date):
    sun_times = sun(location.observer, date=specific_date)
    return sun_times['sunset']

In [100]:
def get_all_dates():
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    return [(start_date + timedelta(days=i)) for i in range((end_date - start_date).days + 1)]

In [67]:
def get_overlap(start1, end1, start2, end2):
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    if overlap_start < overlap_end:
        return overlap_end - overlap_start
    else:
        return timedelta(0)  # Ingen överlapp

In [105]:
from zoneinfo import ZoneInfo
totalt_ljusa_jobb_timmar=0
totalt_ljusa_vakna_timmar=0
dates = get_all_dates()
for date in dates:
    date_in_timzone = date
    sunrise_time = get_sunrise(date_in_timzone)
    sunset_time = get_sunset(date_in_timzone)
    
    work_start_time_today = datetime.combine(date_in_timzone, work_start_time).replace(tzinfo=ZoneInfo("Europe/Stockholm"))
    work_end_time_today = datetime.combine(date_in_timzone, work_end_time).replace(tzinfo=ZoneInfo("Europe/Stockholm"))
    wakeup_time_today = datetime.combine(date_in_timzone, wakeup_time).replace(tzinfo=ZoneInfo("Europe/Stockholm"))
    bed_time_today = datetime.combine(date_in_timzone, bed_time).replace(tzinfo=ZoneInfo("Europe/Stockholm"))

    ljusa_jobb_timmar = get_overlap(sunrise_time,sunset_time, work_start_time_today,work_end_time_today )
    ljusa_vakna_timmar = get_overlap(sunrise_time,sunset_time, wakeup_time_today,bed_time_today )
    totalt_ljusa_jobb_timmar += ljusa_jobb_timmar.total_seconds()
    totalt_ljusa_vakna_timmar += ljusa_vakna_timmar.total_seconds()

print(f"Antal arbetstimmar som är ljusa {totalt_ljusa_jobb_timmar/3600} för året")
print(f"Antal arbetstimmar som är ljusa {totalt_ljusa_jobb_timmar/1314000} per dag")
print(f"Antal vakna som är ljusa {totalt_ljusa_vakna_timmar/3600} för året")
print(f"Antal vakna som är ljusa {totalt_ljusa_vakna_timmar/1314000} per dag")

Antal arbetstimmar som är ljusa 2880.8686775499996 för året
Antal arbetstimmar som är ljusa 7.892790897397259 per dag
Antal vakna som är ljusa 4075.594867145838 för året
Antal vakna som är ljusa 11.166013334646133 per dag


In [123]:

datum = datetime(2024,3,31)
sunrise_time = get_sunrise(datum)
sunset_time = get_sunset(datum)
wakeup_time_today = datetime.combine(datum, wakeup_time).replace(tzinfo=ZoneInfo("Europe/Stockholm"))
bed_time_today = datetime.combine(datum, bed_time).replace(tzinfo=ZoneInfo("Europe/Stockholm"))
ljusa_vakna_timmar = get_overlap(sunrise_time,sunset_time, wakeup_time_today,bed_time_today )

ljusa_vakna_timmar.total_seconds()/3600
# ljusa_vakna_timmar = get_overlap(sunrise_time,sunset_time, wakeup_time_today,bed_time_today )

12.478751464166667

In [133]:
from datetime import datetime, timezone

datum = datetime(2024,3,31)
aware_datetime = datetime(2024, 3, 31, 12, 0, tzinfo=timezone.utc)
aware_datetime_local = datetime(2024, 3, 31, 12, 0, tzinfo=pytz.timezone('Europe/Stockholm'))

print(f"Timezone: {datum.tzinfo}")
print(f"Timezone: {aware_datetime.tzinfo}")

sunrise_time = get_sunrise(aware_datetime_local)

sunrise_time

Timezone: None
Timezone: UTC


datetime.datetime(2024, 3, 31, 6, 16, 11, 361818, tzinfo=<DstTzInfo 'Europe/Stockholm' CEST+2:00:00 DST>)